In [2]:
#Imports
import os
from dateutil import parser
from datetime import datetime, timedelta
import ipywidgets
from geojson import Polygon
import urllib.request, json
import geemap.chart as chart
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from geemap import geojson_to_ee
from ipyleaflet import GeoJSON
import codecs
import json
import time

import geemap.geemap
import ee
ee.Authenticate()
ee.Initialize(project='irrigation-web-app')

from IPython.display import display, HTML
display(HTML("""
<style>
:root { 
    --jp-notebook-max-width: 100% !important;
} 

.lm-Widget.lm-Panel.jp-OutputArea-child{
    display: inline-block;
}

.widget-html-content{
    height: fit-content;
}
</style>
"""))

In [3]:
#Getting Data
# getSentinel2() will get the data with the specified parameters from Sentinel2
# precon: polygonGeoJSON is set
# returns: a FeatureCollection
# Note: Returns the first feature within acceptable cloud levels
def getSentinel2():
    def f(img): 
        cloudBit = img.reduceRegion( #Get median cloud cover
            ee.Reducer.median(), 
            polygonGeoJSON
        ).get('QA60')
        
        #Return
        return ee.Feature(None, { #Empty Feature
            'Kc': img.normalizedDifference(['B8','B4']).multiply(ee.Image.constant(1.37)).subtract(ee.Image.constant(0.086)).reduceRegion(ee.Reducer.mean(), polygonGeoJSON).get('nd'), #Kc
            'time': ee.Date(img.get('system:time_start')), #Set time for graphing
            'cloud': cloudBit, #Set cloud cover for filtering out
            'date': img.date().format("YYYY-MM-dd") #Set date for filtering to 1 a day
        })
    
    cloudSat2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(polygonGeoJSON) \
        .filterDate('2018', str(datetime.today().year)+"12"+"31") \
        .sort('system:time_start')    \
        .map(f) \
        .filter(ee.Filter.eq('cloud', 0)) \
        .distinct('date') \
        .set('dataSet', "Sentinel-2")
    
    return cloudSat2

# quantileS2 will get an array of Kc means, 5th percentiles, and 95th percentiles from Sentinel-2
# precon: polygonGeoJSON is set
# returns: a FeatureCollection
def getQuantileS2():
    def f(img):
        cloudBit = img.reduceRegion( #Get median cloud cover
            ee.Reducer.median(), 
            polygonGeoJSON
        ).get('QA60')
        
        Kcs = img.normalizedDifference(['B8','B4']) \
        .multiply(ee.Image.constant(1.37)).subtract(ee.Image.constant(0.086)) #Kc
            
        return ee.Feature(None, { #Return empty Feature 
            'five': Kcs.reduceRegion(ee.Reducer.percentile([5]), polygonGeoJSON).get('nd'), #5th Percentile
            'ninetyFive': Kcs.reduceRegion(ee.Reducer.percentile([95]), polygonGeoJSON).get('nd'), #95th Percentile
            'mean': Kcs.reduceRegion(ee.Reducer.mean(), polygonGeoJSON).get('nd'), #Average
            'time': ee.Date(img.get('system:time_start')), #Time
            'cloud': cloudBit, #Cloud cover for filtering
            'date': img.date().format("YYYY-MM-dd") #Date for filtering
        })
    
    cloudSat2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(polygonGeoJSON) \
        .filterDate('2018', str(datetime.today().year)+"-12"+"-31") \
        .sort('system:time_start') \
        .map(f) \
        .filter(ee.Filter.eq('cloud', 0))
    
    minMaxRange = cloudSat2.get("date_range")
    return cloudSat2.distinct('date').set({'dataSet': "Landsat", "quantile": "true", "date_range": minMaxRange})

# getLandsat() will get the data with the specified parameters from LandSat
# precon: polygonGeoJSON is set
# returns: a FeatureCollection
def getLandsat():
    def f(img): 
        return ee.Feature(None, { #return empty Feature
            'Kc': img.multiply(0.0000275).add(-0.2).normalizedDifference(['SR_B5','SR_B4']) \
                .multiply(ee.Image.constant(1.37)).subtract(ee.Image.constant(0.086)).reduceRegion(ee.Reducer.mean(), polygonGeoJSON).get('nd'), #Kc
            'time': ee.Date(img.get('system:time_start')), #Time
            'cloud': img.select("ST_CDIST").reduceRegion(ee.Reducer.min(), polygonGeoJSON).get('ST_CDIST'), #Set cloud cover for filtering out
            'date': img.date().format("YYYY-MM-dd") #Set date for filtering to 1 a day
        }) #Format and add dates
    
    landsat8Sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(polygonGeoJSON) \
        .filterDate('2018', str(datetime.today().year)+"12"+"31") \
        .sort('system:time_start') \
        .map(f) \
        .sort('time').distinct('date') \
        .filter(ee.Filter.neq('cloud', 0))
    
    minMaxRange = landsat8Sr.get("date_range") #Data needed for chart
    return landsat8Sr.distinct('date').set({"date_range": minMaxRange, 'dataSet': "Landsat"})

# quantileLandSat will get an array of Kc means, 5th percentiles, and 95th percentiles from Landsat
# precon: polygonGeoJSON is set
# returns: a FeatureCollection
def getQuantileLandSat():
    def f(img): 
        Kcs = img.multiply(0.0000275).add(-0.2).normalizedDifference(['SR_B5','SR_B4']) \
            .multiply(ee.Image.constant(1.37)).subtract(ee.Image.constant(0.086)) #Kc

        return ee.Feature(None, { #Return empty Feature
            'five': Kcs.reduceRegion(ee.Reducer.percentile([5]), polygonGeoJSON).get('nd'), #5th Percentile
            'ninetyFive': Kcs.reduceRegion(ee.Reducer.percentile([95]), polygonGeoJSON).get('nd'), #95th Percentile
            'mean': Kcs.reduceRegion(ee.Reducer.mean(), polygonGeoJSON).get('nd'), #Average
            'time': ee.Date(img.get('system:time_start')), #Time
            'cloud': img.select("ST_CDIST").reduceRegion(ee.Reducer.min(), polygonGeoJSON).get('ST_CDIST'), #Cloud cover for filtering
            'date': img.date().format("YYYY-MM-dd") #Date for filtering
        })
    
    
    
    LandSat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(polygonGeoJSON) \
        .filterDate('2018', str(datetime.today().year)+"12"+"31") \
        .sort('system:time_start') \
        .map(f) \
    .sort('time').distinct('date') \
    .filter(ee.Filter.neq('cloud', 0)) \
    .distinct("date") \
    .set({'dataSet': "Landsat", "quantile": "true"})
    
    return LandSat


In [4]:
#Mapping
# MapDateS2() will map Kc from Sentinel-2
# precon: startDate is set
# returns: nothing, outputs to map
def MapDateS2():
    date = startDate #Get startDate in the correct format
    def f(img):
        cloudBit = img.reduceRegion( #Get cloud cover
            ee.Reducer.median(), 
            polygonGeoJSON
        ).get('QA60')
        
        deltaTime = ee.Date(img.get('system:time_start')).difference(ee.Date(date), 'second').abs()
        return img.normalizedDifference(['B8','B4']).multiply(ee.Image.constant(1.37)).subtract(ee.Image.constant(0.086)).set('deltaTime', deltaTime) \
            .set({ #Additional Info
                'deltaTime': deltaTime, 
                'system:time_start': img.get('system:time_start'), 
                'cloud': cloudBit
            })
    
    cloudSat2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(polygonGeoJSON) \
        .map(f) \
        .filter(ee.Filter.eq('cloud', 0)) \
        .sort('deltaTime')
    
    img = cloudSat2.select("nd").first()
    date = img.date().format("YYYY-MM-dd").getInfo()
    
    i = int(len(ET0[1])/2)
    bottom = 0
    top = len(ET0[1])
    time = 0
    
    while time < len(ET0[1])/2:
        current = ET0[1][i]
        if parser.parse(current) < parser.parse(date):
            bottom = i
        elif parser.parse(current) > parser.parse(date):
            top = i
        elif parser.parse(current) == parser.parse(date):
            break
        
        
        time += 1
        i = int((bottom + top - 0.1) / 2)
    
    ETc = img.multiply(ee.Image.constant(ET0[0][i]))
    
    popupMessage("Mapping closest clear day: " + date) #Inform user of the date used
    minMaxRange = ETc.reduceRegion(ee.Reducer.minMax(), polygonGeoJSON, 10).getInfo() #Get min/max values
    Map.addLayer(ETc, {'palette': ['471d02', '0FF428'], 'min': minMaxRange['nd_min'], 'max': minMaxRange['nd_max']}, 'ETc') #Map image

    return ETc.set({'dataSet': "Sentinel-2", "time": img.get('system:time_start')})

# MapRangeS2() will get the average Kc of all images within the date range 
# precon: startDate is set
# precon: endDate is set
# returns: nothing, outputs to map
def MapRangeS2():
    ETs = [[],[]]

    for i in range(len(ET0[0])): 
        if (startDate < parser.parse(ET0[1][i]) and parser.parse(ET0[1][i]) < endDate): 
            ETs[0].append(ET0[0][i])
            ETs[1].append(ET0[1][i])

    def f(img): 
        cloudBit = img.reduceRegion( #Get cloud cover
            ee.Reducer.median(), 
            polygonGeoJSON
        ).get('QA60')
        
        return img.normalizedDifference(['B8','B4']).multiply(ee.Image.constant(1.37)).subtract(ee.Image.constant(0.086)) \
            .set({
                'time': img.get('system:time_start'), #Set time for graphing
                'cloud': cloudBit, #Set cloud cover for filtering out
                'date': img.date().format("YYYY-MM-dd") #Set date for filtering to 1 a day
            })

    cloudSat2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(polygonGeoJSON) \
        .filterDate(startDate.strftime("%Y-%m-%d"), endDate.strftime("%Y-%m-%d")) \
        .map(f) \
        .filter(ee.Filter.eq('cloud', 0)) \
        .distinct('date')
        
    dates = cloudSat2.aggregate_array("date").getInfo()
    cloudSat2 = cloudSat2.toList(5000000)
    
    Kc = ee.Image.constant(1)
    KcIndex = 0
    ETc = []
    cloudSatSize = cloudSat2.size().getInfo()

    for i in range(cloudSatSize):
        if(dates[i] in ETs[1] and ETs[1].index(dates[i]) != -1):
            KcIndex = i
            Kc = ee.Image(cloudSat2.get(i)).select("nd")
            break
    
    #Kc to ETc cumulative sum

    for i in range(len(ETs[1])):
        if(KcIndex < len(dates) and dates[KcIndex] == ETs[1][i]):
            Kc = ee.Image(cloudSat2.get(KcIndex)).select("nd")
            KcIndex += 1
        
        if(i == 0): x = Kc 
        
        ETcValue = Kc.multiply(ee.Image.constant(ETs[0][i])).cast({"nd": "double"})
        ETc.append(ETcValue)
    
    ETc = ee.ImageCollection(ee.List(ETc)).sum() #Get cumulative sum
    minMaxRange = ETc.reduceRegion(ee.Reducer.minMax(), polygonGeoJSON, 10).getInfo() #Get min/max values
    Map.addLayer(ETc, {'palette': ['471d02', '0FF428'], 'min': minMaxRange['nd_min'], 'max': minMaxRange['nd_max']}, 'ETc') #Map image
    
    return ETc.set({'dataSet': "Sentinel-2"})

# MapDateLandSat() will map Kc from Sentinel-2
# precon: startDate is set
# returns: nothing, outputs to map
def MapDateLandSat(): 
    date = startDate #Get startDate in the correct format
    
    def f(img): 
        deltaTime = ee.Date(img.get('system:time_start')).difference(ee.Date(date), 'second').abs() #Get distance from date to image date
        return img.multiply(0.0000275).add(-0.2).normalizedDifference(['SR_B5','SR_B4']) \
            .multiply(ee.Image.constant(1.37)).subtract(ee.Image.constant(0.086)) \
            .set({ #Additional Info
                'deltaTime': deltaTime, 
                'system:time_start': img.get('system:time_start'),
                'cloud': img.select("ST_CDIST").reduceRegion(ee.Reducer.min(), polygonGeoJSON).get('ST_CDIST')
            })
    

    landsat8Sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(polygonGeoJSON) \
        .map(f) \
        .sort('deltaTime') \
        .filter(ee.Filter.neq('cloud', 0))
        
    img = landsat8Sr.select("nd").first()
    date = img.date().format("YYYY-MM-dd").getInfo()
        
    done = False
    i = int(len(ET0[1])/2)
    bottom = 0
    top = len(ET0[1])
    time = 0
    
    while time < len(ET0[1])/2:
        current = ET0[1][i]
        if parser.parse(current) < parser.parse(date):
            bottom = i
        elif parser.parse(current) > parser.parse(date):
            top = i
        elif parser.parse(current) == parser.parse(date):
            break
        
        time += 1
        i = int((bottom + top - 0.1) / 2)

    
    ETc = img.multiply(ee.Image.constant(ET0[0][i]))
    
    popupMessage("Mapping closest day: " + date) #Inform user of the date used
    minMaxRange = ETc.reduceRegion(ee.Reducer.minMax(), polygonGeoJSON, 10).getInfo() #Get min/max values
    Map.addLayer(ETc, {'palette': ['471d02', '0FF428'], 'min': minMaxRange['nd_min'], 'max': minMaxRange['nd_max']}, 'ETc') #Map the nearest image

    return ETc.set({'dataSet': "LandSat", "time": img.get('system:time_start')})

# MapLandSat() will get the average Kc of all images within the date range 
# precon: startDate is set
# precon: endDate is set
# returns: nothing, outputs to map
def MapRangeLandSat(): 
    ETs = [[],[]]
    for i in range(len(ET0[1])): 
        if(startDate < parser.parse(ET0[1][i]) and parser.parse(ET0[1][i]) < endDate): 
            ETs[0].append(ET0[0][i])
            ETs[1].append(ET0[1][i])
        
    def f(img): 
        return img.multiply(0.0000275).add(-0.2).normalizedDifference(['SR_B5','SR_B4']) \
            .multiply(ee.Image.constant(1.37)).subtract(ee.Image.constant(0.086)) \
            .set({ #Additional Info
                'system:time_start': img.get('system:time_start'),
                'date': img.date().format("YYYY-MM-dd"), #Set date for filtering to 1 a day
                'cloud': img.select("ST_CDIST").reduceRegion(ee.Reducer.min(), polygonGeoJSON).get('ST_CDIST')
            })

    landsat8Sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(polygonGeoJSON) \
        .filter(ee.Filter.greaterThan('system:time_start', startDate.timestamp() * 1000)) \
        .filter(ee.Filter.lessThan('system:time_start', endDate.timestamp() * 1000)) \
        .map(f) \
        .sort('time').distinct('date') \
        .filter(ee.Filter.neq('cloud', 0)) \
        .set('dataSet', "Landsat")
    
    dates = landsat8Sr.aggregate_array("date").getInfo()
    landsat8Sr = landsat8Sr.toList(5000000)
    
    Kc = ee.Image.constant(1)
    KcIndex = 0
    ETc = []
    cloudSatSize = landsat8Sr.size().getInfo()
    
    for i in range(cloudSatSize): 
        if(dates[i] in ETs[1] and ETs[1].index(dates[i]) != -1): 
            KcIndex = i
            Kc = ee.Image(landsat8Sr.get(i)).select("nd")
            break
    
    #Kc to ETc cumulative sum
    for i in range(len(ETs[1])): 
        if(KcIndex + 1 < len(dates) and dates[KcIndex + 1] == ETs[1][i]): 
            KcIndex += 1
            Kc = ee.Image(landsat8Sr.get(KcIndex)).select("nd")
        
        ETcValue = Kc.multiply(ee.Image.constant(ETs[0][i])).cast({"nd": "double"})
        ETc.append(ETcValue)
    
    ETc = ee.ImageCollection(ee.List(ETc)).sum() #Get cumulative sum
    minMaxRange = ETc.reduceRegion(ee.Reducer.minMax(), polygonGeoJSON, 10).getInfo() #Get min/max values
    Map.addLayer(ETc, {'palette': ['471d02', '0FF428'], 'min': minMaxRange['nd_min'], 'max': minMaxRange['nd_max']}, 'ETc') #Map image
    
    return ETc.set({'dataSet': "LandSat"})


In [5]:
#Graphing
# drawChart() will draw a time series graph to the map
# param: ValuesFeatureCollection, a FeatureCollection containing time and data properties
# returns: nothing, draws to map
def drawChart(ValuesFeatureCollection): 
    start = time.time()
    popupMessage("Creating Graph. This may take some time (<10s)")
    
    usedData = ValuesFeatureCollection \
        .filter(ee.Filter.greaterThan('time', ee.Date(startDate.timestamp() * 1000))) \
        .filter(ee.Filter.lessThan('time', ee.Date(endDate.timestamp() * 1000)))

    #Defaults
    dataName = ['Kc']
    first = ee.Feature(usedData.first()).getInfo()
    dataSet = usedData.get('dataSet').getInfo()
    title = 'Kc from ' + dataSet
    colors = ['#1AB4D8']
    
    if (usedData.get("quantile").getInfo() == "true"):
        dataName = ['five','mean','ninetyFive']
        colors = ['#2B7DB5', '#1AB4D8', '#A0F28A']
        title = "Quantile Kc from " + dataSet
    elif ("Kc" not in first["properties"] and "five" not in first["properties"]): #ETc Time Series
        dataName = ['ETc']
        def f(feat): return feat.set({ 'ETc': usedData.filter(ee.Filter.lessThan('time', feat.get('time'))).aggregate_sum('ETc') })
        usedData = usedData.map(f)
        title = 'Cumulative ETc from ' + dataSet
    elif ("Kc" not in first["properties"]): #ETc Quantile
        dataName = ['five','mean','ninetyFive']
        colors = ['#2B7DB5', '#1AB4D8', '#A0F28A']
        
        def f(feat): 
            return feat.set({
                'five': usedData.filter(ee.Filter.lessThan('time', feat.get('time'))).aggregate_sum('five'),
                'mean': usedData.filter(ee.Filter.lessThan('time', feat.get('time'))).aggregate_sum('mean'),
                'ninetyFive': usedData.filter(ee.Filter.lessThan('time', feat.get('time'))).aggregate_sum('ninetyFive')
            })
            
        usedData = usedData.map(f)
        title = 'Cumulative ETc from ' + dataSet

    df = geemap.ee_to_df(usedData)
    plt.figure(figsize=(12, 6)) #, dpi=50 if wanted smaller
    for i in range(len(dataName)):
        plt.plot(pd.to_datetime(df['date']), df[dataName[i]], marker='o', linestyle='-', color=colors[i])
        
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('ETc')
    plt.grid(True)
    plt.xticks(rotation=45)
    chartPanel.clear_output()
    with chartPanel: 
        plt.show()
    
    end = time.time()
    popupMessage("Done in " + time.strftime("%H:%M:%S", time.gmtime(end - start)))

# GraphByYear() will draw a graph of a particular year to the map
# param: ValuesFeatureCollection, a FeatureCollection containing time and data properties
# param: year, the year to use
# returns: nothing, draws to map
# Note: Starts March 15, goes to October 15
def GraphByYear(ValuesFeatureCollection): 
    start = time.time()
    popupMessage("Creating Graph. This may take some time (<5s)")
    date = startDate #Get start date in correct format
    
    usedData = ValuesFeatureCollection \
    .filter(ee.Filter.greaterThan('time', ee.Date(date.replace(month = 3, day = 15).timestamp() * 1000))) \
    .filter(ee.Filter.lessThan('time', ee.Date(date.replace(month = 10, day = 15).timestamp() * 1000)))
    
    def f(feat): 
        return feat.set({ 'ETc': usedData.filter(ee.Filter.lessThan('time', feat.get('time'))).aggregate_sum('ETc') })
    
    usedData = usedData.map(f)

    df = geemap.ee_to_df(usedData)
    plt.figure(figsize=(12, 6)) #, dpi=50 if wanted smaller
    plt.plot(pd.to_datetime(df['date']), df['ETc'], marker='o', linestyle='-', color='#1AB4D8')
    plt.title('Cumulative ETc from ' + usedData.get('dataSet').getInfo())
    plt.xlabel('Date')
    plt.ylabel('ETc')
    plt.grid(True)
    plt.xticks(rotation=45)
    chartPanel.clear_output()
    with chartPanel: 
        plt.show()

    end = time.time()
    popupMessage("Done in " + time.strftime("%H:%M:%S", time.gmtime(end - start)))


In [6]:
#UI Options
# createPanel() will create a panel containing buttons for exporting and graphing Kc from S2 and LandSat
# postcon: the panel will be visible on the map in the top right corner
# Note: Chart of Year, Preview Map, and Export Map all use startDate to choose their time
def createPanel() :     
    # !!! CHART PANEL !!!
    Map.add_widget(chartPanel, position="topleft")
    Map.add_widget(outputPanel, position='bottomright')
    
    # !!! GEOMETRY !!!
    # Create panels to hold the buttons
    panel = ipywidgets.VBox();
    panel1 = ipywidgets.HBox();

     #Submit Geometry
    def f(b) : 
        updateGeometry()
    SubmitBtn = ipywidgets.Button(
        description = "Submit"
    )
    SubmitBtn.on_click(f)

    #Export Geometry
    def f(b) : 
        exportPolygon()
    ExportGeo = ipywidgets.Button(
        description = "Export"
    )
    ExportGeo.on_click(f)

    #Import Geometry
    def f(b) : 
        importPolygon()
    geoJSONUpload.observe(f, 'value')
    
    #Reset Geometry
    def f(b) : 
        resetGeometry()
    ResetGeo = ipywidgets.Button(
        description = "Clear Drawn"
    )
    ResetGeo.on_click(f)

    #Combine and display panels
    panel1.children = (SubmitBtn, ExportGeo, geoJSONUpload, ResetGeo); #Put Export CSV back if needed
    panel.children = (ipywidgets.HTML(value="<span style='width:100%; text-align:center; display:inline-block;'>Geometry</span>"), panel1)
    Map.add_widget(panel, position="topright")


    # !!! DATES !!!
    panel = ipywidgets.HBox();
    panel1 = ipywidgets.VBox();
    panel2 = ipywidgets.VBox();

    panel1.children = (
        ipywidgets.HTML(value="<span style='width:100%; text-align:center; display:inline-block;'>Planting Date</span>"),
        startPicker
    )

    panel2.children = (
        ipywidgets.HTML(value="<span style='width:100%; text-align:center; display:inline-block;'>Harvest Date</span>"),
        endPicker
    )

    panel.children = (panel1, panel2)
    Map.add_widget(panel, position="topright")


    # !!! S2, LANDSAT !!!
    # Create panels to hold the buttons
    panel = ipywidgets.HBox();
    panel1 = ipywidgets.VBox();
    panel2 = ipywidgets.VBox();
    panel3 = ipywidgets.VBox();
    panel4 = ipywidgets.HBox();
    panel5 = ipywidgets.VBox();

    # Sentinel 2
    #Time Series
    def f(b) : 
        drawChart(S2ETc)
    SentinelBtn = ipywidgets.Button(
        description = "Sentinel-2"
    )
    SentinelBtn.on_click(f)

    #Quantile
    def f(b) : drawChart(QuantileS2ETc)
    PercS2 = ipywidgets.Button(
        description = "Sentinel-2"
    )
    PercS2.on_click(f)

    #By Year
    def f(b) : GraphByYear(S2ETc)
    YearS2 = ipywidgets.Button(
        description = "Sentinel-2"
    )
    YearS2.on_click(f)

    #Kc
    def f(b) : drawChart(S2)
    KcS2 = ipywidgets.Button(
        description = "Sentinel-2"
    )
    KcS2.on_click(f)

    #Quantile Kc
    def f(b) : drawChart(quantileS2)
    PercKcS2 = ipywidgets.Button(
        description = "Sentinel-2"
    )
    PercKcS2.on_click(f)
    
    #Preview Range
    def f(b) : MapRangeS2()
    PrevS2Range = ipywidgets.Button(
        description = "Sentinel-2"
    )
    PrevS2Range.on_click(f)

    #Export Range
    def f(b) : exportRange()
    ExportS2Range = ipywidgets.Button(
        description = "Sentinel-2"
    )
    ExportS2Range.on_click(f)

    #Preview Date
    def f(b) : MapDateS2()
    PrevS2 = ipywidgets.Button(
        description = "Sentinel-2"
    )
    PrevS2.on_click(f)

    #Export Date
    def f(b) : exportRaster()
    ExportS2 = ipywidgets.Button(
        description = "Sentinel-2"
    )
    ExportS2.on_click(f)
    
    # LandSat
    #Time Series
    def f(b) : drawChart(LandSatETc)
    LandSatBtn = ipywidgets.Button(
        description = "Landsat"
    )
    LandSatBtn.on_click(f)

    #Quantile
    def f(b) : drawChart(QuantileLandSatETc)
    PercLandSat = ipywidgets.Button(
        description = "Landsat"
    )
    PercLandSat.on_click(f)

    #By Year
    def f(b) : GraphByYear(LandSatETc)
    YearLandSat = ipywidgets.Button(
        description = "Landsat"
    )
    YearLandSat.on_click(f)

    #Kc
    def f(b) : drawChart(LandSat)
    KcLandSat = ipywidgets.Button(
        description = "Landsat"
    )
    KcLandSat.on_click(f)

    #Quantile Kc
    def f(b) : drawChart(quantileLandSat)
    PercKcLandSat = ipywidgets.Button(
        description = "Landsat"
    )
    PercKcLandSat.on_click(f)
    
    #Preview Range
    def f(b) : MapRangeLandSat()
    PrevLandSatRange = ipywidgets.Button(
        description = "Landsat"
    )
    PrevLandSatRange.on_click(f)

    #Export Range
    def f(b) : exportRange()
    ExportLandSatRange = ipywidgets.Button(
        description = "Landsat"
    )
    ExportLandSatRange.on_click(f)

    #Preview Date
    def f(b) : MapDateLandSat()
    PrevLandSat = ipywidgets.Button(
        description = "Landsat"
    )
    PrevLandSat.on_click(f)

    #Export Date
    def f(b) : exportRange()
    ExportLandSat = ipywidgets.Button(
        description = "Landsat"
    )
    ExportLandSat.on_click(f)
    
    #Combine and display panels
    panel1.children = (
        ipywidgets.HTML(value="<span style='width:100%; display:inline-block;'>Time Series</span>"),
        ipywidgets.HTML(value="<span style='width:100%; display:inline-block;'>Quantile</span>"),
        ipywidgets.HTML(value="<span style='width:100%; display:inline-block;'>Chart of Year</span>"),
        ipywidgets.HTML(value="<span style='width:100%; display:inline-block;'>Kc Time Series</span>"),
        ipywidgets.HTML(value="<span style='width:100%; display:inline-block;'>Kc Quantile</span>"),
        ipywidgets.HTML(value="<span style='width:100%; display:inline-block;'>Preview Range</span>"),
        ipywidgets.HTML(value="<span style='width:100%; display:inline-block;'>Export Range</span>"),
        ipywidgets.HTML(value="<span style='width:100%; display:inline-block;'>Preview Date</span>"),
        ipywidgets.HTML(value="<span style='width:100%; display:inline-block;'>Export Date</span>")
    )

    panel2.children = (SentinelBtn, PercS2, YearS2, KcS2, PercKcS2, PrevS2Range, ExportS2Range, PrevS2, ExportS2)
    panel3.children = (LandSatBtn, PercLandSat, YearLandSat, KcLandSat, PercKcLandSat, PrevLandSatRange, ExportLandSatRange, PrevLandSat, ExportLandSat)
    panel4.children = (panel1, panel2, panel3)

    resetRow = ipywidgets.HBox()
    def f(b) :
        chartPanel.clear_output()
        Map.remove_ee_layer('ETc')
        outputPanel.clear_output()

    resetBtn = ipywidgets.Button(
        description = "Reset"
    )
    resetBtn.on_click(f)

    panel6 = ipywidgets.Box(children=[resetBtn], layout=ipywidgets.Layout(display='flex', flex_flow='column', align_items='center', width='100%'))
    resetRow.children = (
        ipywidgets.HTML(value="<span style='width:100%; display:inline-block; white-space:nowrap; height:var(--jp-widgets-inline-height); overflow:hidden;' id='OutputsLabel'>Outputs<br><span style='width:100%; display:inline-block; height:0px; overflow:hidden; line-height:0px;'>Preview Range</span></span>"),
        panel6
    )
    
    panel5.children = (ipywidgets.HTML(value="<span style='width:67%; text-align:right; display:inline-block;'>Datasets</span>"), panel4, resetRow)
    Map.add_widget(panel5, position="topright")

# resetGeometry() will return the geometry to it's initialized position and shape
# returns: nothing
def resetGeometry() : 
    Map.remove_drawn_features()
    Map.remove_ee_layer('ETcArea')
    Map.addLayer(original, {'color': 'red'}, 'ETcArea')
    Map.centerObject(ee.Geometry(original))

# updateGeometry() will update the Kc and Date values to use the currently displayed geometry
# postcon: polygonGeoJSON will be set to the current shape
# postcon: S2 will be a new FeatureCollection from Sentinel-2
# postcon: LandSat will a new FeatureCollection from LandSat
def updateGeometry() : 
    global polygonGeoJSON, S2, quantileS2, LandSat, quantileLandSat, ET0, S2ETc, LandSatETc, QuantileS2ETc, QuantileLandSatETc
    
    if Map.user_rois.size().getInfo() != 0:  
        polygonGeoJSON = Map.user_rois.geometry()
    else:
        polygonGeoJSON = original

    S2 = getSentinel2()
    quantileS2 = getQuantileS2()
    LandSat = getLandsat()
    quantileLandSat = getQuantileLandSat()
    
    import time
    start = time.time()
    popupMessage("Getting ETc data. This may take some time (<20s)")
    ET0 = openMeteo()
    S2ETc = getETcTimeSeries(S2, "Sentinel-2")
    LandSatETc = getETcTimeSeries(LandSat, "Landsat")
    QuantileS2ETc = getETcQuantile(quantileS2, "Sentinel-2")
    QuantileLandSatETc = getETcQuantile(quantileLandSat, "Landsat")
    end = time.time()
    popupMessage("Done in " + time.strftime("%H:%M:%S", time.gmtime(end - start)))

# changeDates() will update the start and end dates for charting
# Note: This will not update the chart. That is up to the user
def changeDates() : 
    global startDate
    global endDate
    
    startDate = datetime(startPicker.value.year, startPicker.value.month, startPicker.value.day) #Set startDate to time integer
    endDate = datetime(endPicker.value.year, endPicker.value.month, endPicker.value.day) #Set endDate to time integer


In [7]:
#Import/Export
# exportRaster() will export the the nearest image to KML with the specified parameters
# precon: startDate must be set | Closest image to this date is used
# precon: polygonGeoJSON must be set | Only pixels in this area will be used
# postcon: file will be exported to downloads
def exportRaster(dataImage):   
    vectors = dataImage.select("nd").sample(**{ #Get individual pixels from image
        'region': polygonGeoJSON,
        'scale': 10,
        'dropNulls': True,
        'geometries': True
    })
    
    outPath = os.path.expanduser("~/Downloads")
    if not os.path.exists(outPath):
        os.makedirs(outPath)
        
    outPath = os.path.join(outPath, "ETc of " + str(dataImage.get("dataSet").getInfo()) + " on " + datetime.fromtimestamp(dataImage.get('time').getInfo() / 1000).strftime("%Y-%m-%d") + ".kml")
    geemap.ee_export_vector(vectors, outPath, selectors=["nd"], verbose=False)
    popupMessage("Done")

# exportRange() will export an avarage Kc off all images within the date range from Sentinel-2
# precon: startDate must be set
# precon: endDate must be set
# precon: polygonGeoJSON must be set | Only pixels in this area will be used
# postcon: file will be exported to downloads
def exportRange(dataImage): 
    vectors = dataImage.select("nd").sample(**{ #Get individual pixels from image
        'region': polygonGeoJSON,
        'scale': 10,
        'dropNulls': True,
        'geometries': True
    })

    outPath = os.path.expanduser("~/Downloads")
    if not os.path.exists(outPath):
        os.makedirs(outPath)
        
    outPath = os.path.join(outPath, "ETc of " + str(dataImage.get("dataSet").getInfo()) + " from " + startDate.strftime("%Y-%m-%d") + " to " + endDate.strftime("%Y-%m-%d") + ".kml")
    geemap.ee_export_vector(vectors, outPath, selectors=["nd"], verbose=False)
    popupMessage("Done")

# exportPolygon() will create a task to export a polygon to the Assets tab
# precon: polygonGeoJSON is set
# postcon: file will be exported to downloads
def exportPolygon(): 
    outPath = os.path.expanduser("~/Downloads")
    if not os.path.exists(outPath):
        os.makedirs(outPath)
        
    outPath = os.path.join(outPath, "PolygonExport.geojson")

    try: 
        geemap.ee_export_vector(polygonGeoJSON, outPath, selectors=[], verbose=False)
    except:
        geemap.ee_export_vector(ee.FeatureCollection(ee.Feature(polygonGeoJSON)), outPath, selectors=[], verbose=False)

    popupMessage("Done")


# exportPolygon() will import a polygon from downloads to the map
# precon: polygonGeoJSON is set
# postcon: the imported polygon is drawn to the map
# Note: file submitted through FileUpload ipywidget
def importPolygon(): 
    popupMessage("Starting")
    value = json.loads(codecs.decode(geoJSONUpload.value[0].content, encoding="utf-8"))

    json_layer = GeoJSON(
        data = value,
        name = "ETcArea"
    )

    global original
    try: 
        original = geojson_to_ee(value).geometry()
    except:
        original = geojson_to_ee(value)

    resetGeometry()


In [8]:
#Open Meteo Stuf
# getCoordinateAverage() will get the average coordinate from polygonGeoJSON
# returns: array, [latitude, longitude]
# Note: Used to feed output to Open-Meteo for ET0 data
def getCoordinateAverage(): 
    coordinates = polygonGeoJSON.coordinates().getInfo()
    coordSum = [0, 0]

    if(isinstance(coordinates[0][0], list)): 
        coordLength = 0
        for coords in coordinates[0]:
            coordLength += len(coords)
            # for coord in coords:
            coordSum[0] += coords[0]
            coordSum[1] += coords[1]
        
        coordSum[0] /= coordLength
        coordSum[1] /= coordLength
    else:
        for coord in coordinates[0]:
            coordSum[0] += coord[0]
            coordSum[1] += coord[1]

        coordSum[0] /= len(coordinates[0])
        coordSum[1] /= len(coordinates[0])
    
    return coordSum

# openMeteo will get the ET0 values at the average position specified by polygonGeoJSON
# returns: a lists of lists in format [[ET0], [date]]
# Notes: There may be gaps due to null values
def openMeteo(): 
    start = "2018-01-01";
    end = (datetime.today() - timedelta(days=2)).strftime("%Y-%m-%d");

    coordSum = getCoordinateAverage()
    
    with urllib.request.urlopen("https://api.open-meteo.com/v1/forecast?latitude=" + str(coordSum[1]) + "&longitude=" + str(coordSum[0]) + "&daily=et0_fao_evapotranspiration&timezone=America%2FDenver&past_days=2") as url:
        forecast = json.load(url)

    with urllib.request.urlopen("https://archive-api.open-meteo.com/v1/archive?latitude=" + str(coordSum[1]) + "&longitude=" + str(coordSum[0]) + "&start_date=" + start + "&end_date=" + end + "&daily=et0_fao_evapotranspiration&timezone=America%2FDenver") as url:
        previous = json.load(url)

    list = [
        previous["daily"]["et0_fao_evapotranspiration"] + (forecast["daily"]["et0_fao_evapotranspiration"]), 
        previous["daily"]["time"] + (forecast["daily"]["time"])
    ]
    
    usedList = [[], []]
    
    for i in range(len(list[0])): 
        if(list[0][i] != None): 
            usedList[0].append(list[0][i])
            usedList[1].append(list[1][i])
    
    return usedList


# getETcTimeSeries() will get the ETc from S2 time series and openMeteo data
# param: ValuesFeatureCollection, a FeatureCollection containing time and data properties
# returns: a FeatureCollection
# Note: due to use of getInfo(), call may be slow
def getETcTimeSeries(ValuesFeatureCollection, title): 
    #Initialization
    usedData = ValuesFeatureCollection \
        .toList(5000000).getInfo()
    
    Kc = 1
    KcIndex = 0
    ETc = []
    for i in range(len(usedData)): 
        if (usedData[i]["properties"]["date"] in ET0[1] and ET0[1].index(usedData[i]["properties"]["date"]) != -1): 
            KcIndex = i
            Kc = usedData[i]["properties"]["Kc"]
            break
    
    #Kc to ETc values
    for i in range(len(ET0[1])): 
        if(KcIndex + 1 < len(usedData) and usedData[KcIndex + 1]["properties"]["date"] == ET0[1][i]): 
            KcIndex += 1
            Kc = usedData[KcIndex]["properties"]["Kc"]

        
        ETcValue = (Kc * ET0[0][i]) #Get current ETc plus previous
        
        ETc.append(ee.Feature(None, { #Empty Feature
            'ETc': ETcValue, #Kc
            'time': parser.parse(ET0[1][i]), #Set time for graphing
            'date': ET0[1][i] #Set date for filtering to 1 a day
        }))
    
    return ee.FeatureCollection(ee.List(ETc)).set("dataSet", title) #Convert to FeatureCollection

# getETcQuantile() will get the ETc from quantile S2 and openMeteo data
# param: ValuesFeatureCollection, a FeatureCollection containing time and data properties
# returns: a FeatureCollection
# Note: due to use of getInfo(), call may be slow
def getETcQuantile(ValuesFeatureCollection, title): 
    #Initialization
    usedData = ValuesFeatureCollection \
        .toList(50000).getInfo()
    
    Kc5 = 1
    Kcm = 1
    Kc95 = 1
    KcIndex = 0
    ETc = []
    for i in range(len(usedData)): 
        if(usedData[i]["properties"]["date"] in ET0[1] and ET0[1].index(usedData[i]["properties"]["date"]) != -1): 
            KcIndex = i
            Kc5 = usedData[KcIndex]["properties"]["five"]
            Kcm = usedData[KcIndex]["properties"]["mean"]
            Kc95 = usedData[KcIndex]["properties"]["ninetyFive"]
            break
    
    #Kc to ETc cumulative sum
    for i in range(len(ET0[1])): 
        if(KcIndex + 1 < len(usedData) and usedData[KcIndex + 1]["properties"]["date"] == ET0[1][i]): 
            KcIndex += 1
            Kc5 = usedData[KcIndex]["properties"]["five"]
            Kcm = usedData[KcIndex]["properties"]["mean"]
            Kc95 = usedData[KcIndex]["properties"]["ninetyFive"]
        
        #Get current ETc
        ETcValue5 = (Kc5 * ET0[0][i]) 
        ETcValuem = (Kcm * ET0[0][i])
        ETcValue95 = (Kc95 * ET0[0][i])
        
        ETc.append(ee.Feature(None, { #Empty Feature
            'five': ETcValue5, #Kc
            'mean': ETcValuem,
            'ninetyFive': ETcValue95,
            'time': parser.parse(ET0[1][i]), #Set time for graphing
            'date': ET0[1][i]
        }))
    
    return ee.FeatureCollection(ee.List(ETc)).sort('time').set('dataSet', title) #Convert to FeatureCollection

def popupMessage(message): 
    with outputPanel:
        print(message)


In [9]:
#Initialize map
os.environ["HYBRID"] = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
Map = geemap.Map(height='700pt', width="100%", fullscreen_ctrl=True, toolbar_ctrl=False, layer_ctrl=False)
Map.add_basemap("HYBRID")

#Initialize values
original = ee.Geometry.Polygon([[(-112.75576440080,49.690961914171),(-112.75117245897,49.690961914171),(-112.75117245897,49.693738071755),(-112.75576440080,49.693738071755),(-112.75576440080,49.690961914171)]])
polygonGeoJSON = original
outputPanel = ipywidgets.Output(layout={'border': '1px solid black'})

S2 = getSentinel2()
quantileS2 = getQuantileS2()
LandSat = getLandsat()
quantileLandSat = getQuantileLandSat()

start = time.time()
popupMessage("Getting ETc data. This may take some time (<20s)")
ET0 = openMeteo()
S2ETc = getETcTimeSeries(S2, "Sentinel-2")
LandSatETc = getETcTimeSeries(LandSat, "Landsat")
QuantileS2ETc = getETcQuantile(quantileS2, "Sentinel-2")
QuantileLandSatETc = getETcQuantile(quantileLandSat, "Landsat")
end = time.time()
popupMessage("Done in " + time.strftime("%H:%M:%S", time.gmtime(end - start)))

In [10]:
#Initialization
# Dates
while True:
    startDate = datetime.today()
    endDate = datetime.today()
    
    if(startDate.month > 4):
        startDate = datetime(int(startDate.year), 4, 1)
    else:
        startDate = datetime(int(startDate.year)-1, 4, 1)
        endDate = datetime(int(endDate.year)-1, 8, 30)
        break

    if(startDate.month > 8):
        endDate = datetime(int(endDate.year), 8, 30)
    else:
        endDate = (endDate + timedelta(days=6)).replace(hour=0, minute=0, second=0, microsecond=0)
        
    break


def f(change): 
    changeDates()
startPicker = ipywidgets.DatePicker(value=startDate)
startPicker.observe(f, names='value')
endPicker = ipywidgets.DatePicker(value=endDate)
endPicker.observe(f, 'value')

# Chart
chartPanel = ipywidgets.Output()
geoJSONUpload = ipywidgets.FileUpload(accept='.geojson', multiple=False)

# Map Initialization
os.environ["HYBRID"] = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
Map = geemap.Map(height='700pt', width="100%", fullscreen_ctrl=True, toolbar_ctrl=False, layer_ctrl=False)
Map.add_basemap("HYBRID")

resetGeometry()
createPanel()

# !!! TESTING AREA !!! 



Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…